In [0]:
CREATE OR REFRESH LIVE TABLE dim_date_da AS
SELECT * FROM LIVE.silver_date;

In [0]:
CREATE OR REFRESH LIVE TABLE dim_location_da AS
WITH src AS (
  SELECT
    fi.street_address,
    fi.zip_code,
    g.city,
    g.state,
    AVG(fi.latitude)  AS latitude,
    AVG(fi.longitude) AS longitude
  FROM LIVE.silver_food_inspection_dallas fi
  LEFT JOIN LIVE.silver_geo g
    ON fi.zip_code = g.zip_code
  WHERE fi.zip_code IS NOT NULL
  GROUP BY
    fi.street_address,
    fi.zip_code,
    g.city,
    g.state
)
SELECT
  DENSE_RANK() OVER (
    ORDER BY
      zip_code,
      street_address,
      city,
      state
  ) AS location_key,
  street_address,
  city,
  state,
  zip_code,
  latitude,
  longitude
FROM src;

In [0]:
CREATE OR REFRESH LIVE TABLE dim_inspection_type_da AS
SELECT * FROM LIVE.silver_inspection_type

---------------------------------------------------------------------------
ParseException                            Traceback (most recent call last)
File <command-7378559087892557>, line 1
----> 1 get_ipython().run_cell_magic('sql', '', 'CREATE OR REFRESH LIVE TABLE dim_inspection_type_da AS\n')

File /databricks/python/lib/python3.12/site-packages/IPython/core/interactiveshell.py:2541, in InteractiveShell.run_cell_magic(self, magic_name, line, cell)
   2539 with self.builtin_trap:
   2540     args = (magic_arg_s, cell)
-> 2541     result = fn(*args, **kwargs)
   2543 # The code below prevents the output from being displayed
   2544 # when using magics with decorator @output_can_be_silenced
   2545 # when the last Python token in the expression is a ';'.
   2546 if getattr(fn, magic.MAGIC_OUTPUT_CAN_BE_SILENCED, False):

File /databricks/python_shell/lib/dbruntime/sql_magic/sql_magic.py:194, in SqlMagic.sql(self, line, cell)
    188 except BaseException as e:
    189     self.driver

In [0]:
CREATE OR REFRESH LIVE TABLE dim_violation_da AS
SELECT * FROM LIVE.silver_violation

In [0]:
CREATE OR REFRESH LIVE TABLE fact_inspection_da AS
WITH src AS (
  SELECT
    trim(restaurant_name)                       AS restaurant_name,
    inspection_type,
    CAST(inspection_date AS DATE)              AS inspection_dt,
    CAST(inspection_score AS INT)              AS inspection_score,
    CAST(total_violation_score AS INT)         AS total_violation_score,
    street_address,
    CAST(zip_code AS STRING)                   AS zip_code,
    latitude,
    longitude,
    load_dt,
    source_file_path,
    source_file_name
  FROM LIVE.silver_food_inspection_dallas
),

biz AS (
  SELECT
    business_key,
    restaurant_name
  FROM (
    SELECT
      business_key,
      restaurant_name,
      ROW_NUMBER() OVER (
        PARTITION BY restaurant_name
        ORDER BY effective_start_dt DESC
      ) AS rn
    FROM LIVE.dim_business_da_scd2_v2
  )
  WHERE rn = 1          
),

dt AS (
  SELECT
    date_key,
    full_date
  FROM LIVE.dim_date_da
),

insp_type AS (
  SELECT
    inspection_type_key,
    inspection_type
  FROM LIVE.dim_inspection_type_da
),

loc AS (
  SELECT
    location_key,
    street_address,
    zip_code,
    ROW_NUMBER() OVER (
      PARTITION BY street_address, zip_code
      ORDER BY location_key
    ) AS rn
  FROM LIVE.dim_location_da
),

joined AS (
  SELECT
    d.date_key                 AS date_key,
    b.business_key             AS business_da_key,
    l.location_key             AS location_key,
    it.inspection_type_key     AS inspection_type_key,

    s.inspection_score,
    s.total_violation_score,
    s.load_dt,
    s.source_file_path,
    s.source_file_name,

    s.restaurant_name,
    s.inspection_dt            AS inspection_date,
    s.inspection_type,
    s.zip_code
  FROM src s

  LEFT JOIN dt d
    ON d.full_date = s.inspection_dt

  LEFT JOIN biz b
  ON trim(s.restaurant_name) = trim(b.restaurant_name)

  LEFT JOIN loc l
    ON s.street_address = l.street_address
   AND s.zip_code      = l.zip_code
   AND l.rn            = 1

  LEFT JOIN insp_type it
    ON s.inspection_type = it.inspection_type
),

dedup AS (
  SELECT
    *,
    ROW_NUMBER() OVER (
      PARTITION BY restaurant_name, inspection_date, inspection_type, zip_code
      ORDER BY load_dt DESC    
    ) AS rn
  FROM joined
)

SELECT
  ROW_NUMBER() OVER (
    ORDER BY restaurant_name, inspection_date, inspection_type, zip_code
  )                          AS inspection_da_key,

  date_key,
  business_da_key,
  location_key,
  inspection_type_key,

  inspection_score,
  total_violation_score

FROM dedup
WHERE rn = 1;

In [0]:
CREATE OR REFRESH LIVE TABLE bridge_inspection_violation_da AS

WITH fact_natural AS (
  SELECT
    f.inspection_da_key,
    trim(b.restaurant_name)         AS restaurant_name,
    d.full_date                     AS inspection_date,
    it.inspection_type              AS inspection_type,
    cast(l.zip_code AS string)      AS zip_code
  FROM LIVE.fact_inspection_da f
  LEFT JOIN LIVE.dim_business_da_scd2_v2 b
    ON f.business_da_key = b.business_key
  LEFT JOIN LIVE.dim_date_da d
    ON f.date_key = d.date_key
  LEFT JOIN LIVE.dim_inspection_type_da it
    ON f.inspection_type_key = it.inspection_type_key
  LEFT JOIN LIVE.dim_location_da l
    ON f.location_key = l.location_key
),

src AS (
  SELECT
    fn.inspection_da_key,
    s.violation_description_1,
    s.violation_description_2,
    s.violation_description_3,
    s.violation_description_4,
    s.violation_description_5,
    s.violation_description_6,
    s.violation_description_7,
    s.violation_description_8,
    s.violation_description_9,
    s.violation_description_10,
    s.violation_description_11,
    s.violation_description_12,
    s.violation_description_13,
    s.violation_description_14,
    s.violation_description_15,
    s.violation_description_16,
    s.violation_description_17,
    s.violation_description_18,
    s.violation_description_19,
    s.violation_description_20,
    s.violation_description_21,
    s.violation_description_22,
    s.violation_description_23,
    s.violation_description_24,
    s.violation_description_25
  FROM fact_natural fn
  JOIN LIVE.silver_food_inspection_dallas s
    ON upper(trim(fn.restaurant_name))  = upper(trim(s.restaurant_name))
   AND fn.inspection_date               = CAST(s.inspection_date AS DATE)
   AND upper(trim(fn.inspection_type))  = upper(trim(s.inspection_type))
   AND fn.zip_code                      = cast(s.zip_code as string)
),

exploded AS (
  SELECT
    inspection_da_key,
    posexplode(
      array(
        nullif(trim(violation_description_1),  ''),
        nullif(trim(violation_description_2),  ''),
        nullif(trim(violation_description_3),  ''),
        nullif(trim(violation_description_4),  ''),
        nullif(trim(violation_description_5),  ''),
        nullif(trim(violation_description_6),  ''),
        nullif(trim(violation_description_7),  ''),
        nullif(trim(violation_description_8),  ''),
        nullif(trim(violation_description_9),  ''),
        nullif(trim(violation_description_10), ''),
        nullif(trim(violation_description_11), ''),
        nullif(trim(violation_description_12), ''),
        nullif(trim(violation_description_13), ''),
        nullif(trim(violation_description_14), ''),
        nullif(trim(violation_description_15), ''),
        nullif(trim(violation_description_16), ''),
        nullif(trim(violation_description_17), ''),
        nullif(trim(violation_description_18), ''),
        nullif(trim(violation_description_19), ''),
        nullif(trim(violation_description_20), ''),
        nullif(trim(violation_description_21), ''),
        nullif(trim(violation_description_22), ''),
        nullif(trim(violation_description_23), ''),
        nullif(trim(violation_description_24), ''),
        nullif(trim(violation_description_25), '')
      )
    ) AS (pos, violation_description)
  FROM src
),

flat AS (
  SELECT
    inspection_da_key,
    pos + 1 AS sequence_n,  
    violation_description,
    regexp_extract(violation_description, '(\\d+)', 1) AS violation_code
  FROM exploded
  WHERE violation_description IS NOT NULL
),

joined AS (
  SELECT
    f.inspection_da_key,
    v.violation_code,
    f.sequence_n
  FROM flat f
  JOIN LIVE.dim_violation_da v
    ON v.violation_code = f.violation_code
  WHERE f.violation_code IS NOT NULL AND f.violation_code <> ''
)

SELECT DISTINCT
  inspection_da_key,
  violation_code,
  sequence_n
FROM joined;